In [70]:
from src.param_parser import parameter_parser
import json
import random
import networkx as nx
import numpy as np
import torch

from src.gam import GAMTrainer, GAM
from src.utils import create_features

In [20]:
class ARGS(object):
    def __init__(self):
        self.train_graph_folder = './input/train/'
        self.test_graph_folder = './input/test'
        self.prediction_path = './output/erdos_predictions.csv'
        self.log_path = './logs/erdos_gam_logs.json'
        self.epochs = 10
        self.step_dimensions = 32
        self.combined_dimensions = 64
        self.batch_size = 3
        self.time = 10
        self.repetitions = 10
        self.gamma = 0.99
        self.learning_rate = 0.001
        self.weight_decay = 10 ** -5

In [21]:
args = ARGS()
trainer = GAMTrainer(args=args)

100%|██████████| 50/50 [00:00<00:00, 5860.26it/s]



The number of graph classes is: 3.



In [68]:
# figure out stupid JSON loading logic

# understanding process_graph
# 1. get dictionary with keys (target, edges, labels, inverse_labels), values as expected
data = json.load(open(trainer.training_graphs[0]))
# 2. create_features produces a NetworkX graph with its node features
# features is a 2D tensor of one-hot labels for each node in graph (see create_features)
graph, features = create_features(data, trainer.model.identifiers)
# 3. pick a random node
node = random.choice(list(graph.nodes))
attention_loss = 0
# 4. iterate for some time (limited by budget)
for t in range(args.time):
    # 5. call forward pass of GAM
    predictions, node, attention_score = trainer.model(data, graph, features, node)
    # in forward call of GAM, only instance of graph is the forward call to GAM.step_block (type: StepNetworkLayer)
    # in forward call of GAM.step_block, graph only appears in step_block.make_step
    # in call to make_step, two NetworkX functions are used:
        # nx.neighbors
            # can be replaced by torch.where(adj[node_id])?
        # in step_block.sample_node_labels, which needs graph.nodes
            # can be replaced by torch.range(adj.shape[0])?


In [73]:
# understanding create_features

identifiers = [1]
# 1. make networkx graph without features 
# can be replaced with filling in a torch tensor of zeros
graph = nx.from_edgelist(data["edges"])
# 2. fill in node feature matrix
# features are one-hot vectors - not sure how important a feature matrix is, but we can possibly
# replace it with a vector of one's whose length is the number of nodes (same as only having one
# label)
features = []
for node in graph.nodes():
    features.append([1.0 if data["labels"][str(node)] == i else 0.0 for i in range(len(identifiers))])
# 3. features cast to np.array and then torch.Tensor
# features = np.array(features, dtype=np.float32)
features = torch.tensor(features)

In [74]:
# understanding process_batch
# typical training loop

In [ ]:
# understanding fit
# instead of manually creating batches, we use procecss_batch, but otherwise there's no strange logic I see

In [22]:
trainer.training_graphs

['./input/train/20.json',
 './input/train/36.json',
 './input/train/41.json',
 './input/train/16.json',
 './input/train/6.json',
 './input/train/7.json',
 './input/train/17.json',
 './input/train/40.json',
 './input/train/37.json',
 './input/train/21.json',
 './input/train/47.json',
 './input/train/10.json',
 './input/train/26.json',
 './input/train/30.json',
 './input/train/31.json',
 './input/train/27.json',
 './input/train/1.json',
 './input/train/50.json',
 './input/train/11.json',
 './input/train/46.json',
 './input/train/2.json',
 './input/train/28.json',
 './input/train/12.json',
 './input/train/45.json',
 './input/train/32.json',
 './input/train/24.json',
 './input/train/49.json',
 './input/train/48.json',
 './input/train/25.json',
 './input/train/33.json',
 './input/train/44.json',
 './input/train/13.json',
 './input/train/29.json',
 './input/train/3.json',
 './input/train/34.json',
 './input/train/8.json',
 './input/train/22.json',
 './input/train/18.json',
 './input/train/38

In [19]:
trainer.model.training_graphs

AttributeError: 'GAM' object has no attribute 'training_graphs'